In [ ]:
#@markdown <center><h3><h2>ShortGPT</h2>Install Requirements. This is 1 Click Run App to create Short Videos</h3></center><br>
# 📦 Install ImageMagick and ffmpeg
!sudo apt-get install imagemagick
!sudo apt-get install ffmpeg

# 📝 Fix Imagemagick policy.xml bug in linux using MoviePy
!sed -i '/<policy domain="path" rights="none" pattern="@\*"/d' /etc/ImageMagick-6/policy.xml

import os
if not os.path.exists('/content/ShortGPT'):
  # 📂 Clone the ShortGPT repository
  !git clone https://github.com/enterprisium/ShortGPT.git
  %cd /content/ShortGPT/
else:
  %cd /content/ShortGPT/
  # If the directory already exists, just pull the latest version
  !git pull
# 🚀 Change directory to ShortGPT

# 🛠️ Install required Python packages
!pip install -r requirements.txt

In [ ]:
# Import necessary modules from ShortGPT
from shortGPT.config.api_db import ApiKeyManager, ApiProvider
from shortGPT.config.asset_db import AssetDatabase, AssetType
from shortGPT.engine.facts_short_engine import FactsShortEngine
from shortGPT.audio.edge_voice_module import EdgeTTSVoiceModule
from shortGPT.audio.coqui_voice_module import CoquiVoiceModule
from shortGPT.audio.eleven_voice_module import ElevenLabsVoiceModule  # Add this line to import ElevenLabsVoiceModule
from shortGPT.config.languages import Language, EDGE_TTS_VOICENAME_MAPPING
from shortGPT.config.languages import LANGUAGE_ACRONYM_MAPPING, Language
import random

# Load API keys from environment variables or hardcoded values
OPENAI_API = ""  # @param {type:"string"}
ELEVEN_LABS_API = ""  # @param {type:"string"}
PEXELS_API = ""  # @param {type:"string"}

# Configuration parameters
NUM_OF_SHORTS = 1  # @param {type:"integer"}
TYPE_OF_SHORTS = "Astrophotography and Space Facts: Combine stunning visuals of the night sky with fascinating space facts, catering to the curiosity of viewers interested in astronomy."  # @param {type:"string"}
TTS_ENGINE = "ElevenLabs"  # @param ["Edge TTS", "ElevenLabs", "Coqui TTS"]
LANGUAGE = "English"  # @param ["English", "Spanish", "French", "Arabic", "German", "Polish", "Italian", "Portuguese", "Afrikaans", "Amharic", "Azerbaijani", "Bulgarian", "Bengali", "Bosnian", "Catalan", "Czech", "Welsh", "Danish", "Greek", "Estonian", "Persian", "Finnish", "Filipino", "Galician", "Gujarati", "Hebrew", "Hindi", "Croatian", "Hungarian", "Indonesian", "Icelandic", "Japanese", "Javanese", "Georgian", "Kazakh", "Khmer", "Kannada", "Korean", "Lao", "Lithuanian", "Latvian", "Macedonian", "Malayalam", "Mongolian", "Marathi", "Malay", "Maltese", "Myanmar", "Norwegian", "Nepali", "Dutch", "Norwegian Bokmål", "Norwegian Nynorsk", "Pashto", "Romanian", "Russian", "Sinhala", "Slovak", "Slovenian", "Somali", "Albanian", "Serbian", "Sundanese", "Swedish", "Swahili", "Tamil", "Telugu", "Thai", "Turkish", "Ukrainian", "Urdu", "Uzbek", "Vietnamese", "Chinese", "Zulu"]
VOICE_NAME = "Antoni"  # @param ['Rachel', 'Drew', 'Clyde', 'Paul', 'Domi', 'Dave', 'Fin', 'Sarah', 'Antoni', 'Thomas', 'Charlie', 'George', 'Emily', 'Elli', 'Callum', 'Patrick', 'Harry', 'Liam', 'Dorothy', 'Josh', 'Arnold', 'Charlotte', 'Matilda', 'Matthew', 'James', 'Joseph', 'Jeremy', 'Michael', 'Ethan', 'Gigi', 'Freya', 'Grace', 'Daniel', 'Lily', 'Serena', 'Adam', 'Nicole', 'Bill', 'Jessie', 'Sam', 'Glinda', 'Giovanni', 'Mimi', 'Claribel Dervla', 'Daisy Studious', 'Gracie Wise', 'Tammie Ema', 'Alison Dietlinde', 'Ana Florence', 'Annmarie Nele', 'Asya Anara', 'Brenda Stern', 'Gitta Nikolina', 'Henriette Usha', 'Sofia Hellen', 'Tammy Grit', 'Tanja Adelina', 'Vjollca Johnnie', 'Andrew Chipper', 'Badr Odhiambo', 'Dionisio Schuyler', 'Royston Min', 'Viktor Eka', 'Abrahan Mack', 'Adde Michal', 'Baldur Sanjin', 'Craig Gutsy', 'Damien Black', 'Gilberto Mathias', 'Ilkin Urbano', 'Kazuhiko Atallah', 'Ludvig Milivoj', 'Suad Qasim', 'Torcull Diarmuid', 'Viktor Menelaos', 'Zacharie Aimilios', 'Nova Hogarth', 'Maja Ruoho', 'Uta Obando', 'Barbora MacLean', 'Alexandra Hisakawa', 'Alma María', 'Rosemary Okafor', 'Ige Behringer', 'Filip Traverse', 'Damjan Chapman', 'Wulf Carlevaro', 'Aaron Dreschner', 'Kumar Dahl', 'Eugenio Mataracı', 'Ferran Simen', 'Xavier Hayasaka', 'Luis Moray', 'Marcos Rudaski']
USE_IMAGE = 10  # @param {type:"integer"}
WATERMARK = ""  # @param {type:"string"}
BG_VIDEO = "Ski gameplay"  # @param {type:"string"}
BG_MUSIC = "Music joakim karud dreams"  # @param {type:"string"}
FONT_STYLE = Anta-Regular.ttf  # @param ["Arial-Bold", "Roboto-Regular", "Times New Roman"] # New parameter for font style

# Set the API keys for different services
ApiKeyManager.set_api_key(ApiProvider.OPENAI, OPENAI_API)
ApiKeyManager.set_api_key(ApiProvider.ELEVEN_LABS, ELEVEN_LABS_API)
ApiKeyManager.set_api_key(ApiProvider.PEXELS, PEXELS_API)

def select_voice_module(tts_engine, language, voice):
    if tts_engine == "Edge TTS":
        voice_name = EDGE_TTS_VOICENAME_MAPPING[Language[language.upper()]]['male']  # Defaulting to male if specific voice not handled
        return EdgeTTSVoiceModule(voice_name)
    elif tts_engine == "ElevenLabs":
        return ElevenLabsVoiceModule(api_key=ELEVEN_LABS_API, voiceName=voice)
    elif tts_engine == "Coqui TTS":
        language_code = LANGUAGE_ACRONYM_MAPPING.get(Language[language.upper()], 'en')  # Default to 'en' if not found
        return CoquiVoiceModule(voiceName=voice, language=language_code)
    else:
        raise ValueError("Unsupported TTS Engine")

# Correctly select the voice module based on the TTS_ENGINE parameter
try:
    voice_module = select_voice_module(TTS_ENGINE, LANGUAGE, VOICE_NAME)
except ValueError as e:
    print(f"Error selecting voice module: {e}")
    # Fallback to a default voice module if error occurs
    voice_module = EdgeTTSVoiceModule(EDGE_TTS_VOICENAME_MAPPING[Language.ENGLISH]['male'])

# Configure the Content Engine with FONT_STYLE parameter
content_engine = FactsShortEngine(
    voiceModule=voice_module,
    facts_type=TYPE_OF_SHORTS,
    background_video_name=BG_VIDEO,
    background_music_name=BG_MUSIC,
    num_images=USE_IMAGE,
    watermark=WATERMARK,
    language=Language[LANGUAGE.upper()],
)

# Generate Content
try:
    for step_num, step_logs in content_engine.makeContent():
        print(f"Step {step_num}: {step_logs}")
except Exception as e:
    print(f"An error occurred: {e}")

# Get Video Output Path
print(content_engine.get_video_output_path())

In [ ]:
#@markdown <center><h2>Script To Video</h2>Make Sure To Select ElevenLabs Or Edge TTS Only</h3></center><br>
# Import necessary modules from the ShortGPT framework
from shortGPT.config.api_db import ApiKeyManager, ApiProvider
from shortGPT.config.asset_db import AssetDatabase, AssetType
from shortGPT.engine.content_video_engine import ContentVideoEngine
from shortGPT.config.languages import Language, EDGE_TTS_VOICENAME_MAPPING
from shortGPT.audio.edge_voice_module import EdgeTTSVoiceModule
from shortGPT.audio.eleven_voice_module import ElevenLabsVoiceModule
from shortGPT.audio.coqui_voice_module import CoquiVoiceModule

# Load API keys from hardcoded values - Consider using environment variables for better security
OPENAI_KEY = ""  # @param {type:"string"}
ELEVEN_KEY = ""  # @param {type:"string"}
PEXELS_KEY = ""  # @param {type:"string"}

# Set API Keys
ApiKeyManager.set_api_key(ApiProvider.OPENAI, OPENAI_KEY)
ApiKeyManager.set_api_key(ApiProvider.ELEVEN_LABS, ELEVEN_KEY)
ApiKeyManager.set_api_key(ApiProvider.PEXELS, PEXELS_KEY)

# Add Assets
AssetDatabase.add_remote_asset('chill music', AssetType.BACKGROUND_MUSIC, "https://www.youtube.com/watch?v=uUu1NcSHg2E")

# Function to select the voice module based on user preference
def select_voice_module(tts_engine, language, voice):
    if tts_engine == "Edge TTS":
        voice_name = EDGE_TTS_VOICENAME_MAPPING[Language[language.upper()]]['male']  # Defaulting to male if specific voice not handled
        return EdgeTTSVoiceModule(voice_name)
    elif tts_engine == "ElevenLabs":
        if language not in eleven_labs_languages or voice not in eleven_labs_voices:
            raise ValueError(f"Selected language '{language}' or voice '{voice}' is not supported by ElevenLabs. Supported languages are: {', '.join(eleven_labs_languages)}. Supported voices are: {', '.join(eleven_labs_voices)}.")
        return ElevenLabsVoiceModule(api_key=ELEVEN_KEY, voiceName=voice)
    elif tts_engine == "Coqui TTS":
        if language not in coqui_tts_languages or voice not in coqui_tts_voices:
            raise ValueError(f"Selected language '{language}' or voice '{voice}' is not supported by Coqui TTS. Supported languages are: {', '.join(coqui_tts_languages)}. Supported voices are: {', '.join(coqui_tts_voices)}.")
        return CoquiVoiceModule(voiceName=voice, language=language)
    else:
        raise ValueError("Unsupported TTS Engine")

# Configure the Voice Module
TEXT_TO_SPEECH = "Edge TTS"  # @param ["Edge TTS", "ElevenLabs", "Coqui TTS"]
LANGUAGE = "English"  # @param ["English", "Spanish", "French", "Arabic", "German", "Polish", "Italian", "Portuguese"]
VOICE = "Damjan Chapman"  # @param ['Rachel', 'Drew', 'Clyde', 'Paul', 'Domi', 'Dave', 'Fin', 'Sarah', 'Antoni', 'Thomas', 'Charlie', 'George', 'Emily', 'Elli', 'Callum', 'Patrick', 'Harry', 'Liam', 'Dorothy', 'Josh', 'Arnold', 'Charlotte', 'Matilda', 'Matthew', 'James', 'Joseph', 'Jeremy', 'Michael', 'Ethan', 'Gigi', 'Freya', 'Grace', 'Daniel', 'Lily', 'Serena', 'Adam', 'Nicole', 'Bill', 'Jessie', 'Sam', 'Glinda', 'Giovanni', 'Mimi', 'Claribel Dervla', 'Daisy Studious', 'Gracie Wise', 'Tammie Ema', 'Alison Dietlinde', 'Ana Florence', 'Annmarie Nele', 'Asya Anara', 'Brenda Stern', 'Gitta Nikolina', 'Henriette Usha', 'Sofia Hellen', 'Tammy Grit', 'Tanja Adelina', 'Vjollca Johnnie', 'Andrew Chipper', 'Badr Odhiambo', 'Dionisio Schuyler', 'Royston Min', 'Viktor Eka', 'Abrahan Mack', 'Adde Michal', 'Baldur Sanjin', 'Craig Gutsy', 'Damien Black', 'Gilberto Mathias', 'Ilkin Urbano', 'Kazuhiko Atallah', 'Ludvig Milivoj', 'Suad Qasim', 'Torcull Diarmuid', 'Viktor Menelaos', 'Zacharie Aimilios', 'Nova Hogarth', 'Maja Ruoho', 'Uta Obando', 'Barbora MacLean', 'Alexandra Hisakawa', 'Alma María', 'Rosemary Okafor', 'Ige Behringer', 'Filip Traverse', 'Damjan Chapman', 'Wulf Carlevaro', 'Aaron Dreschner', 'Kumar Dahl', 'Eugenio Mataracı', 'Ferran Simen', 'Xavier Hayasaka', 'Luis Moray', 'Marcos Rudaski']
try:
    voice_module = select_voice_module(TEXT_TO_SPEECH, LANGUAGE, VOICE)
except ValueError as e:
    print(f"Error selecting voice module: {e}")
    # Fallback to a default voice module if error occurs
    voice_module = EdgeTTSVoiceModule(EDGE_TTS_VOICENAME_MAPPING[Language.ENGLISH]['male'])

# Prepare the script
SCRIPT = "\"Artificial intelligence (AI) is revolutionizing our world in surprising ways. Robots and virtual assistants help us in our daily tasks and simplify our lives. In medicine, AI enables more accurate diagnoses and advances in treatments. In the automotive industry, autonomous vehicles are changing the way we get around. However, questions arise about the impact on employment and the ethics of its use. Despite the challenges, AI promises an exciting future full of possibilities. Are we ready to embrace this technological advancement?"  # @param {type:"string"}

# Configure Content Engine
content_engine = ContentVideoEngine(voice_module, SCRIPT, background_music_name='chill music', language=Language[LANGUAGE.upper()])

# Generate Content with error handling
try:
    for step_num, step_logs in content_engine.makeContent():
        print(f"Step {step_num}: {step_logs}")
except Exception as e:
    print(f"Error generating content: {e}")

# Get Video Output Path
print("Video Output Path:", content_engine.get_video_output_path())

In [ ]:
# Import necessary modules from the ShortGPT framework
from shortGPT.config.api_db import ApiKeyManager, ApiProvider
from shortGPT.config.asset_db import AssetDatabase, AssetType
from shortGPT.engine.content_video_engine import ContentVideoEngine
from shortGPT.config.languages import Language, EDGE_TTS_VOICENAME_MAPPING, LANGUAGE_ACRONYM_MAPPING
from shortGPT.audio.edge_voice_module import EdgeTTSVoiceModule
from shortGPT.audio.eleven_voice_module import ElevenLabsVoiceModule
from shortGPT.audio.coqui_voice_module import CoquiVoiceModule

# Load API keys from hardcoded values - Consider using environment variables for better security
OPENAI_KEY = ""  # @param {type:"string"}
ELEVEN_KEY = ""  # @param {type:"string"}
PEXELS_KEY = ""  # @param {type:"string"}

# Set API Keys
ApiKeyManager.set_api_key(ApiProvider.OPENAI, OPENAI_KEY)
ApiKeyManager.set_api_key(ApiProvider.ELEVEN_LABS, ELEVEN_KEY)
ApiKeyManager.set_api_key(ApiProvider.PEXELS, PEXELS_KEY)

# Add Assets
AssetDatabase.add_remote_asset('chill music', AssetType.BACKGROUND_MUSIC, "https://www.youtube.com/watch?v=uUu1NcSHg2E")

# Function to select the voice module based on user preference
def select_voice_module(tts_engine, language, voice):
    if tts_engine == "Edge TTS":
        voice_name = EDGE_TTS_VOICENAME_MAPPING[Language[language.upper()]]['male']  # Defaulting to male if specific voice not handled
        return EdgeTTSVoiceModule(voice_name)
    elif tts_engine == "ElevenLabs":
        # Assuming eleven_labs_languages and eleven_labs_voices are defined elsewhere
        return ElevenLabsVoiceModule(api_key=ELEVEN_KEY, voiceName=voice)
    elif tts_engine == "Coqui TTS":
        language_code = LANGUAGE_ACRONYM_MAPPING.get(Language[language.upper()], 'en')  # Default to 'en' if not found
        return CoquiVoiceModule(voiceName=voice, language=language_code)
    else:
        raise ValueError("Unsupported TTS Engine")

# Configure the Voice Module
TEXT_TO_SPEECH = "Coqui TTS"  # @param ["Edge TTS", "ElevenLabs", "Coqui TTS"]
LANGUAGE = "Hindi"  # @param ["English", "Hindi", "French", "Arabic", "Urdu", "Roman Urdu", "Italian", "Portuguese"]
VOICE = "Damjan Chapman"  # @param ['Rachel', 'Drew', 'Clyde', 'Paul', 'Domi', 'Dave', 'Fin', 'Sarah', 'Antoni', 'Thomas', 'Charlie', 'George', 'Emily', 'Elli', 'Callum', 'Patrick', 'Harry', 'Liam', 'Dorothy', 'Josh', 'Arnold', 'Charlotte', 'Matilda', 'Matthew', 'James', 'Joseph', 'Jeremy', 'Michael', 'Ethan', 'Gigi', 'Freya', 'Grace', 'Daniel', 'Lily', 'Serena', 'Adam', 'Nicole', 'Bill', 'Jessie', 'Sam', 'Glinda', 'Giovanni', 'Mimi', 'Claribel Dervla', 'Daisy Studious', 'Gracie Wise', 'Tammie Ema', 'Alison Dietlinde', 'Ana Florence', 'Annmarie Nele', 'Asya Anara', 'Brenda Stern', 'Gitta Nikolina', 'Henriette Usha', 'Sofia Hellen', 'Tammy Grit', 'Tanja Adelina', 'Vjollca Johnnie', 'Andrew Chipper', 'Badr Odhiambo', 'Dionisio Schuyler', 'Royston Min', 'Viktor Eka', 'Abrahan Mack', 'Adde Michal', 'Baldur Sanjin', 'Craig Gutsy', 'Damien Black', 'Gilberto Mathias', 'Ilkin Urbano', 'Kazuhiko Atallah', 'Ludvig Milivoj', 'Suad Qasim', 'Torcull Diarmuid', 'Viktor Menelaos', 'Zacharie Aimilios', 'Nova Hogarth', 'Maja Ruoho', 'Uta Obando', 'Barbora MacLean', 'Alexandra Hisakawa', 'Alma María', 'Rosemary Okafor', 'Ige Behringer', 'Filip Traverse', 'Damjan Chapman', 'Wulf Carlevaro', 'Aaron Dreschner', 'Kumar Dahl', 'Eugenio Mataracı', 'Ferran Simen', 'Xavier Hayasaka', 'Luis Moray', 'Marcos Rudaski']
# Prepare the script
SCRIPT = "SCRIPT = \"एक समय की बात है, जब पृथ्वी पर जीवन की शुरुआत हुई थी। इस ग्रह पर एक अनोखी प्रजाति ने अपने कदम रखे, जिसे हम मानव कहते हैं। यह कहानी है उस मानव जाति की, जिसने समय के साथ अपने जीवन को नई दिशा दी। शुरुआत में, मानव जंगलों और गुफाओं में रहते थे। उनका जीवन संघर्षों से भरा था। उन्होंने आग की खोज की, जिससे उनके जीवन में एक नई रोशनी आई। आग ने उन्हें ठंड से बचाया और खाना पकाने का तरीका सिखाया। समय के साथ, मानव ने खेती की शुरुआत की। उन्होंने अनाज उगाना शुरू किया और पशुओं को पालना सीखा। इससे उनके जीवन में स्थिरता आई और समाज का विकास हुआ। फिर आया सभ्यताओं का युग। मानव ने नदियों के किनारे बस्तियाँ बसाईं और विशाल साम्राज्यों की नींव रखी। उन्होंने लिपि का आविष्कार किया, जिससे ज्ञान का संचार हुआ। विज्ञान, कला, और साहित्य के क्षेत्र में उन्होंने अद्भुत कार्य किए। लेकिन मानव इतिहास में संघर्ष भी रहे। युद्ध, महामारी, और प्राकृतिक आपदाओं ने मानवता की परीक्षा ली। फिर भी, मानव ने कभी हार नहीं मानी। उन्होंने हर बार उठकर एक नई शुरुआत की। आज, हम एक ऐसे युग में हैं, जहाँ तकनीकी ने हमारे जीवन को बदल दिया है। हमने चाँद पर कदम रखा है और अंतरिक्ष की गहराइयों को छूने की कोशिश की है। यह कहानी हमें याद दिलाती है कि मानव जाति का इतिहास संघर्ष और विजय की कहानी है। हमारी यात्रा अद्भुत रही है, और हमारा भविष्य अभी भी लिखा जा रहा है। हमें अपने पूर्वजों की विरासत को आगे बढ़ा"   # @param {type:"string"}

try:
    voice_module = select_voice_module(TEXT_TO_SPEECH, LANGUAGE, VOICE)
except ValueError as e:
    print(f"Error selecting voice module: {e}")
    # Fallback to a default voice module if error occurs
    voice_module = EdgeTTSVoiceModule(EDGE_TTS_VOICENAME_MAPPING[Language.ENGLISH]['male'])

# Prepare the script
SCRIPT = "\"Artificial intelligence (AI) is revolutionizing our world in surprising ways. Robots and virtual assistants help us in our daily tasks and simplify our lives. In medicine, AI enables more accurate diagnoses and advances in treatments. In the automotive industry, autonomous vehicles are changing the way we get around. However, questions arise about the impact on employment and the ethics of its use. Despite the challenges, AI promises an exciting future full of possibilities. Are we ready to embrace this technological advancement?"

# Configure Content Engine
content_engine = ContentVideoEngine(voice_module, SCRIPT, background_music_name='chill music', language=Language[LANGUAGE.upper()])

# Generate Content with error handling
try:
    for step_num, step_logs in content_engine.makeContent():
        print(f"Step {step_num}: {step_logs}")
except Exception as e:
    print(f"Error generating content: {e}")

# Get Video Output Path
print("Video Output Path:", content_engine.get_video_output_path())

 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
Error selecting voice module: Expecting value: line 1 column 1 (char 0)
Step 1: Current step (1 / 10) : _generateTempAudio
Step 2: Current step (2 / 10) : _speedUpAudio
Step 3: Current step (3 / 10) : _timeCaptions
Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



100%|███████████████████████████████████████| 139M/139M [00:03<00:00, 39.6MiB/s]


Detected language: English


100%|██████████| 396/396 [00:00<00:00, 858.28frames/s]


Step 4: Current step (4 / 10) : _generateVideoSearchTerms
Expecting value: line 1 column 1 (char 0)
not the right format
Expecting value: line 1 column 1 (char 0)
not the right format
Expecting value: line 1 column 1 (char 0)
not the right format
Expecting value: line 1 column 1 (char 0)
not the right format
Expecting value: line 1 column 1 (char 0)
not the right format
Expecting value: line 1 column 1 (char 0)
not the right format
Expecting value: line 1 column 1 (char 0)
not the right format
Step 5: Current step (5 / 10) : _generateVideoUrls
Step 6: Current step (6 / 10) : _chooseBackgroundMusic
Step 7: Current step (7 / 10) : _prepareBackgroundAssets
Step 8: Current step (8 / 10) : _prepareCustomAssets
Step 9: Current step (9 / 10) : Preparing rendering assets...


...

Moviepy - Building video .editing_assets/general_video_assets/d497348cbfcb418d835889a4/rendered_video.mp4.
MoviePy - Writing audio in rendered_videoTEMP_MPY_wvf_snd.mp4




100%|██████████| 68.0/68.0 [09:58<00:00, 8.80s/iB]

100%|██████████| 68.0/68.0 [09:58<00:00, 8.80s/iB]

MoviePy - Done.
Moviepy - Writing video .editing_assets/general_video_assets/d497348cbfcb418d835889a4/rendered_video.mp4





100%|██████████| 68.0/68.0 [10:05<00:00, 8.90s/iB]

100%|██████████| 68.0/68.0 [10:05<00:00, 8.90s/iB]

Moviepy - Done !
Moviepy - video ready .editing_assets/general_video_assets/d497348cbfcb418d835889a4/rendered_video.mp4
Step 10: Current step (10 / 10) : _addMetadata
Video Output Path: videos/2024-02-28_21-37-54 - The Future of AI Boon or Bane .mp4
